In [1]:
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import dill as pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
def load_sentence_embeddings(model='DistilBERT', features_path='features/', filename='sentence_embeddings'):
    if model == 'DistilBERT':
        tst_df = pd.concat([pickle.load(open(os.path.join(features_path, file), 'rb')) for file in os.listdir(features_path) if filename in file])
        return tst_df

In [3]:
tst_df = load_sentence_embeddings()

In [4]:
tst_df.shape

(22332, 2)

In [5]:
tst_df.head()

,sentence_embeddings,label
0,"[-0.21086546778678894, -0.005486507900059223, ...",0
1,"[-0.3340323865413666, 0.1280461549758911, -0.6...",0
2,"[-0.4694300889968872, -0.14065003395080566, -0...",0
3,"[-0.19011789560317993, 0.16007745265960693, -0...",0
4,"[-0.5151359438896179, -0.09672432392835617, -0...",0


In [6]:
features = np.array(tst_df.sentence_embeddings.tolist())
labels = tst_df['label']

In [11]:
model = Sequential()
model.add(Dense(12, input_shape=(768, ), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [12]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
model.fit(features, labels, epochs=150, batch_size=10)

Epoch 1/150
2234/2234 [==============================] - 6s 2ms/step - loss: 0.1672 - accuracy: 0.9306
Epoch 2/150
2234/2234 [==============================] - 5s 2ms/step - loss: 0.0873 - accuracy: 0.9673
Epoch 3/150
2234/2234 [==============================] - 5s 2ms/step - loss: 0.0717 - accuracy: 0.9737
Epoch 4/150
2234/2234 [==============================] - 5s 2ms/step - loss: 0.0640 - accuracy: 0.9764
Epoch 5/150
2234/2234 [==============================] - 6s 3ms/step - loss: 0.0612 - accuracy: 0.9772
Epoch 6/150
2234/2234 [==============================] - 5s 2ms/step - loss: 0.0566 - accuracy: 0.9788
Epoch 7/150
2234/2234 [==============================] - 5s 2ms/step - loss: 0.0522 - accuracy: 0.9795
Epoch 8/150
2234/2234 [==============================] - 5s 2ms/step - loss: 0.0495 - accuracy: 0.9812
Epoch 9/150
2234/2234 [==============================] - 5s 2ms/step - loss: 0.0452 - accuracy: 0.9828
Epoch 10/150
2234/2234 [==============================] - 5s 2ms/step - l

2234/2234 [==============================] - 5s 2ms/step - loss: 0.0056 - accuracy: 0.9982
Epoch 80/150
2234/2234 [==============================] - 5s 2ms/step - loss: 0.0038 - accuracy: 0.9988
Epoch 81/150
2234/2234 [==============================] - 5s 2ms/step - loss: 0.0062 - accuracy: 0.9979
Epoch 82/150
2234/2234 [==============================] - 6s 2ms/step - loss: 0.0026 - accuracy: 0.9991
Epoch 83/150
2234/2234 [==============================] - 5s 2ms/step - loss: 0.0046 - accuracy: 0.9983
Epoch 84/150
2234/2234 [==============================] - 5s 2ms/step - loss: 0.0042 - accuracy: 0.9986
Epoch 85/150
2234/2234 [==============================] - 5s 2ms/step - loss: 0.0058 - accuracy: 0.9983
Epoch 86/150
2234/2234 [==============================] - 5s 2ms/step - loss: 0.0041 - accuracy: 0.9989
Epoch 87/150
2234/2234 [==============================] - 5s 2ms/step - loss: 0.0052 - accuracy: 0.9983
Epoch 88/150
2234/2234 [==============================] - 5s 2ms/step - loss:

In [15]:
_, accuracy = model.evaluate(features, labels, verbose=0)

In [16]:
accuracy

0.9997313022613525

In [18]:
predictions = (model.predict(features) > 0.5).astype(int)


for i in range(5):
    print('{1} (expected {2})'.format(features[i].tolist(), predictions[i], labels[i]))

698/698 [==============================] - 1s 2ms/step
[0] (expected 0    0
0    0
0    0
Name: label, dtype: int64)
[0] (expected 1    0
1    0
1    0
Name: label, dtype: int64)
[0] (expected 2    0
2    0
2    0
Name: label, dtype: int64)
[0] (expected 3    0
3    0
3    0
Name: label, dtype: int64)
[0] (expected 4    0
4    0
4    0
Name: label, dtype: int64)
